In [ ]:
import matplotlib,aplpy,numpy,math,astropy
from astropy.wcs import WCS
from astropy.io import fits
import matplotlib.pyplot as plt
from astroquery.vizier import Vizier
from astropy.table import QTable, Table
from astropy import units
from astropy.coordinates import SkyCoord
import common_functions as cf

In [ ]:
font = {'size'   : 14, 'family' : 'serif', 'serif' : 'cm'}
plt.rc('font', **font)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['lines.linewidth'] = 1
plt.rcParams['axes.linewidth'] = 1

In [ ]:
c = 299792.458 #km/s
#HI line rest frequency
f0 = 1.420405751 #GHz
#Hubble constant
H0 = 70. #km/s/Mpc
h = H0/100.
#Solar mass
Msol = 1.99E30 #kg
#Proton mass
mp = 1.67E-27 #kg
#Parsec
pc = 3.086E16 #m

In [ ]:
GBT_axis = {7: 'f', 10: 'f', 15: 'f', 16: 'f', 23: 'f', 25: 'f', 26: 'f', 30: 'f', 31: 'f', 37: 'f', 
            40: 'v', 48: 'v', 58: 'v', 68: 'f', 79: 'v', 88: 'f', 90: 'f', 91: 'f', 92: 'f', 93: 'f', 97: 'f', 100: 'f'}
VLA_HCGs = [2, 7, 10, 15, 16, 19, 22, 23, 25, 26, 30, 31, 33, 37, 38, 40, 47, 48, 49, 54, 56, 57, 58, 59, 61, 62, 68, 71, 79, 88, 89, 90, 91, 92, 93, 95, 96, 97, 100]
GBT_point = QTable([[92, 48, 58, 88, 91, 37, 68, 79, 93, 7, 16, 25, 31, 97, 23, 30, 100, 15, 26, 10, 33, 90, 40],
                    ['22h35m57.500s','10h37m45.600s','11h42m10.700s','20h52m24.000s','22h09m10.400s','09h13m35.600s','13h53m26.000s','15h59m11.800s','23h15m24.200s',
                     '00h39m23.900s','02h09m31.300s','03h20m43.700s','05h01m38.300s','23h47m22.900s','03h07m06.400s','04h36m28.600s','00h01m20.700s','02h07m38.900s',
                     '03h21m54.200s','01h26m13.000s','05h10m47.900s','22h02m05.600s','09h38m54.500s'],
                    ['+33d57m35.00s','-27d04m49.00s','+10d18m21.00s','-05d45m00.00s','-27d47m45.00s','+30d00m51.00s','+40d19m08.00s','+20d45m30.00s','+18d58m59.00s',
                     '+00d52m41.00s','-10d09m30.00s','-01d03m06.00s','-04d15m25.00s','-02d19m34.00s','-09d35m07.00s','-02d49m56.00s','+13d07m57.00s','+02d08m17.00s',
                     '-13d38m45.00s','+34d42m41.00s','+18d02m05.00s','-31d58m00.00s','-04d51m07.00s']],
                   names = ['HCG', 'ra', 'dec'])
GBT_int_limits = {7: [3970.,4550.],
                  10: [4750.,5500.],
                  15: [6100.,7000.],
                  16: [3600.,4180.],
                  23: [4330.,5200.],
                  25: [6010.,6670.],
                  26: [9220.,9750.],
                  30: [4250.,4760.],
                  31: [3870.,4270.],
                  37: [6200.,7400.],
                  40: [6150.,7000.],
                  48: [2200.,2600.],
                  58: [5880.,6990.],
                  68: [2100.,2500.],
                  79: [3910.,4940.],
                  88: [5890.,6290.],
                  90: [2000.,2500.],
                  91: [6750.,7450.],
                  92: [5610.,6770.],
                  93: [4500.,5200.],
                  97: [6150.,7300.],
                  100: [5000.,5600.]}
GBT_point.add_index('HCG')

In [ ]:
HCGs = Table.read('tables/HCGs.vo', format='votable')
HCGs.add_index('HCG')

HCGs['logMHI_GBT'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['logMHI_VLA'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['logMHI_VLA_inGBTbeam'] = numpy.zeros(len(HCGs))+numpy.nan
HCGs['VLA_beam_maj'] = numpy.zeros(len(HCGs))
HCGs['VLA_beam_maj'].unit = units.arcsec
HCGs['VLA_beam_min'] = numpy.zeros(len(HCGs))
HCGs['VLA_beam_min'].unit = units.arcsec
HCGs['chan_wid'] = numpy.zeros(len(HCGs))
HCGs['chan_wid'].unit = units.km/units.s
HCGs['VLA_rms'] = numpy.zeros(len(HCGs))
HCGs['VLA_rms'].unit = units.mJy/units.beam 
HCGs['VLA_NHI'] = numpy.zeros(len(HCGs))
HCGs['VLA_NHI'].unit = 1/(units.cm)**2
HCGs['VLA_nHI'] = numpy.zeros(len(HCGs))
HCGs['VLA_nHI'].unit = units.solMass/(units.pc)**2

HCG_mems = Table.read('tables/HCG_members.vo', format='votable')
HCG_mems.add_index('name')

In [ ]:
for HCG in VLA_HCGs:
    mask = 'SoFiA_masks/HCG{0}/HCG{0}_mask.fits'.format(HCG)
    pbcube = 'SoFiA_masks/HCG{0}/HCG{0}_HI.pbcor.fits'.format(HCG)
    flatcube = 'SoFiA_masks/HCG{0}/HCG{0}_HI.fits'.format(HCG)
    src_cat = 'SoFiA_masks/HCG{0}/HCG{0}_cat.xml'.format(HCG)
    
    print("HCG{}:".format(HCG))
    
    try:
        mask,mask_ra,mask_dec,mask_vel = cf.read_fitscube(mask,mask=True)
        flat_cube,cube_ra,cube_dec,cube_vel,bmaj,bmin,pa,beam_factor,cube_dx,cube_dy,cube_dv = cf.read_fitscube(flatcube,True,True)
        cube,cube_ra,cube_dec,cube_vel,bmaj,bmin,pa,beam_factor,cube_dx,cube_dy,cube_dv = cf.read_fitscube(pbcube,True,True)
        src_cat = Table.read(src_cat)
    except FileNotFoundError:
        print("Mask, cube, or catalogue file not found for HCG {}.".format(HCG))
        continue
        
    pbcor = numpy.nansum(cube*mask,axis=0)/numpy.nansum(flat_cube*mask,axis=0)
        
    rms = numpy.mean(src_cat['rms'])
    print("rms = {} mJy/beam".format(1000.*rms))
    print('Beam: {0}" x {1}"'.format(numpy.round(bmaj,decimals=1),numpy.round(bmin,decimals=1)))
    n_HI = (rms*abs(cube_dv)*2.356E-7)/(numpy.pi*bmaj*bmin*(numpy.pi/(2.*180.*3600.))**2.)
    print("3-sigma n_HI = {} Msol/pc^2".format(3.*n_HI))
    N_HI = n_HI*Msol/(mp*(pc*100.)**2.)
    print("3-sigma N_HI = {} cm^-2".format(3.*N_HI))
    err_VLA_spec = rms*numpy.sqrt(numpy.nansum(numpy.nansum(mask*pbcor,axis=1),axis=1)/beam_factor)
    
    HCGs.loc[HCG]['VLA_rms'] = numpy.round(rms*1000.,decimals=2)
    HCGs.loc[HCG]['VLA_NHI'] = numpy.round(N_HI,decimals=2)
    HCGs.loc[HCG]['VLA_nHI'] = n_HI
    HCGs.loc[HCG]['chan_wid'] = numpy.round(numpy.absolute(cube_dv),decimals=1)
    HCGs.loc[HCG]['VLA_beam_maj'] = numpy.round(bmaj,decimals=1)
    HCGs.loc[HCG]['VLA_beam_min'] = numpy.round(bmin,decimals=1)
        
    masked_cube = numpy.multiply(cube,mask)
    VLA_spec = numpy.nansum(numpy.nansum(masked_cube,axis=1),axis=1)/beam_factor
    
    HCGs.loc[HCG]['logMHI_VLA'] = numpy.round(numpy.log10(numpy.nansum(VLA_spec)*abs(cube_dv)*235600.*HCGs.loc[HCG]['dist']**2.),decimals=2)
    
    GBT_spec_exists = False
    if HCG in GBT_axis.keys():
        GBT_spec_exists = True
        
    if GBT_spec_exists:
        gain_fac = 1.65
        if GBT_axis[HCG] == 'f':
            GBT_freq, GBT_spec = numpy.loadtxt('GBT_spectra/h{}.ascii'.format(HCG),unpack=True)
            GBT_vel = c*(f0 - GBT_freq)/GBT_freq
            GBT_spec = GBT_spec/gain_fac
        else:
            GBT_vel, GBT_spec = numpy.loadtxt('GBT_spectra/h{}_v.ascii'.format(HCG),unpack=True)
            GBT_spec = GBT_spec/gain_fac
        inx = numpy.where(numpy.logical_and(GBT_vel > GBT_int_limits[HCG][0],GBT_vel < GBT_int_limits[HCG][1]))[0]
        HCGs.loc[HCG]['logMHI_GBT'] = numpy.round(numpy.log10(numpy.nansum(GBT_spec[inx])*((max(GBT_vel[inx])-min(GBT_vel[inx]))*(1./(len(inx)-1.)))*235600.*HCGs.loc[HCG]['dist']**2.),decimals=2)
        
    try:
        HCG_cz = HCGs.loc[HCG]['z']*c
    except KeyError:
        print("No redshift for HCG {} in HCG catalogue.".format(HCG))
        continue
        
    if GBT_spec_exists:
        GBT_pointing = SkyCoord(GBT_point.loc[HCG]['ra'], GBT_point.loc[HCG]['dec'], frame='icrs', equinox='J2000')
        #Make empty array of pixel weights
        pix_wts = numpy.zeros(numpy.shape(cube[0]))
        #Set values for the length of the axes
        leni,lenj = numpy.shape(pix_wts)

        #Calculate the beam response in every pixel
        for i in range(leni):
            #Define the pixel positions of a column of pixels in the WCS
            pix_pos = SkyCoord(numpy.repeat(cube_ra[i],len(cube_dec)), cube_dec, frame='icrs', equinox='J2000', unit='deg')
            #Calculate the separations from the pointing centre
            sep = GBT_pointing.separation(pix_pos)
            #Weight by the beam response (2.3548 is the factor to convert HPBW to STD)
            #Convolution of two Gaussians is a Gaussian  with sig^2 = sig1^2 + sig2^2
            fwhm = numpy.sqrt(9.1**2. - ((bmaj+bmin)*0.5/60.)**2.)
            pix_wts[i] = cf.gaussian(sep.arcmin,0.,fwhm/2.3548)

        #Apply the pixel weighting to the masked cube and sum to get the weighted spectrum
        VLA_spec_wt = numpy.nansum(numpy.nansum(masked_cube*pix_wts,axis=1),axis=1)/beam_factor
        
        HCGs.loc[HCG]['logMHI_VLA_inGBTbeam'] = numpy.round(numpy.log10(numpy.nansum(VLA_spec_wt)*abs(cube_dv)*235600.*HCGs.loc[HCG]['dist']**2.),decimals=2)
    
    #Plot the spectra
    plt.figure(figsize=(8,7))
    if GBT_spec_exists:
        n_smo = int(numpy.round(10./((max(GBT_vel)-min(GBT_vel))/(len(GBT_vel)-1.))))
        GBT_spec_smo = numpy.convolve(GBT_spec,numpy.ones(n_smo)/float(n_smo),mode='same')
        plt.step(GBT_vel,1000.*GBT_spec_smo,c='grey',lw=1,where='mid',label='GBT')
        plt.step(cube_vel,1000.*VLA_spec_wt,c='b',lw=1,where='mid',label='VLA (GBT beam)')
    plt.fill_between(cube_vel,1000.*(VLA_spec+err_VLA_spec),1000.*(VLA_spec-err_VLA_spec),step='mid',color='k',lw=0,alpha=0.25)
    plt.step(cube_vel,1000.*VLA_spec,c='k',lw=2,where='mid',label='VLA (Full)')
    plt.axhline(rms*1000.,c='k',lw=1,ls='--')
    plt.axhline(-rms*1000.,c='k',lw=1,ls='--')
    plt.axhline(c='k',lw=1)

    plt.xlim(HCG_cz-750.,HCG_cz+750.)
    VLA_max = 1000.*max(numpy.where(numpy.logical_and(cube_vel > HCG_cz-750., cube_vel < HCG_cz+750.),VLA_spec,0.))
    if GBT_spec_exists:
        GBT_max = 1000.*max(numpy.where(numpy.logical_and(GBT_vel > HCG_cz-750., GBT_vel < HCG_cz+750.),GBT_spec_smo,0.))
        plt.ylim(-0.2*max(GBT_max,VLA_max),1.2*max(GBT_max,VLA_max))
    plt.ylim(-0.2*VLA_max,1.3*VLA_max)
    plt.annotate(s='HCG {}'.format(HCG),xy=[0.85,0.9],xycoords='axes fraction')
    
    #Mark redshifts of members
    mems = HCG_mems[HCG_mems['HCG'] == HCG]
    vert_offset = 0.
    for mem_name in mems['name']:
        inx = mem_name.find('HCG')
        if inx != -1:
            if HCG < 10:
                name_str = mem_name[inx+4:]
            else:
                name_str = mem_name[inx+5:]
        else:
            name_str = mem_name
        
        plt.axvline(HCG_mems.loc[mem_name]['Vel'],lw=1,ls='dotted',c='k')
        plt.annotate(name_str,xy=[HCG_mems.loc[mem_name]['Vel']-5.,(0.95 - 0.04*vert_offset)*(1.3*VLA_max)],horizontalalignment='right')
        vert_offset += 1.

    #Label axes
    plt.xlabel(r'$v_\mathrm{opt}/\mathrm{km\,s^{-1}}$')
    plt.ylabel(r'Flux Density/mJy')
    
    #Save
    plt.savefig('spectra/HCG{}_spec.pdf'.format(HCG))
    plt.savefig('spectra/HCG{}_spec.png'.format(HCG))
    
    plt.show()

In [ ]:
HCGs['HIdef_VLA'] = numpy.round(HCGs['logMHI_pred'] - HCGs['logMHI_VLA'],decimals=2)
HCGs['HIdef_GBT'] = numpy.round(HCGs['logMHI_pred'] - HCGs['logMHI_GBT'],decimals=2)

In [ ]:
HCGs

In [ ]:
HCGs.write('tables/HCGs_global_HI.vo',format='votable',overwrite=True)
HCGs.write('tables/HCGs_global_HI.fits',format='fits',overwrite=True)
HCGs.write('tables/HCGs_global_HI.ascii',format='ascii.fixed_width_two_line',overwrite=True)